# Homomorphic Encryption

This notebook performs computations from "Fully Homomorphic Encryption over the Integers" 

First, let's do some basic Python imports and initialization.

In [426]:
from math import ceil, floor, log
import random
random.seed(1234)

Next, we setup our parameters for the R-LWE bootstrap encryption.

We want a small example, so the parameters are way too small to provide any real security.

In [ ]:
rho = 2
rho_prime = 3
nu = 9
gamma = 9
tau = 6

print ('Public key integer bit size:', gamma)
print ('Secret key bit size:', nu)
print ('Noise bit size:', rho)
print ('Secondary noise parameter:', rho_prime)
print ('Number of integers in public key:', tau + 1)
print ('Maximum error:', ((2**rho) - 1) * (tau + 1) + ((2**rho_prime) - 1))

Public key integer bit size: 9
Secret key bit size: 9
Noise bit size: 2
Secondary noise parameter: 3
Number of integers in public key: 7
Maximum error: 28


Next, we define some functions to help us sample random values and calculate smaller, potentially negative remainders.

In [428]:
def D(p):
    q = random.randint(0, int(floor((2**gamma) / p)))
    r = random.randint(-(2**rho) + 1, (2**rho) - 1)
    x = p * q + r
    return x

def rem(a, b):
    q = int(round(float(a) / b))
    return a - q * b

We generate a secret key, a random odd integer.

In [ ]:
p = random.randint(2**(nu-2), (2**(nu-1))) * 2 + 1
print ('Secret key', p)

Secret key 505


We generate a public key from our secret key: a list of integers that are zero mod p with some random noise added.

In [ ]:
while True:
    xi = [D(p) for _ in range(tau + 1)]
    maxx = xi[0]
    maxi = 0
    for i in range(1, len(xi)):
        x = xi[i]
        if x > maxx:
            maxx = x
            maxi = i
    xi[0], xi[maxi] = xi[maxi], xi[0]
    # x0 must be odd
    if xi[0] % 2 == 0:
        continue
    # x0 % p must be even
    if rem(xi[0], p) % 2 == 1:
        continue
    # avoid edge case of x0 divisible by p
    if xi[0] % p == 0:
        continue
    break
print ('Public key', xi)


Public key [507, 503, 503, 502, -3, 2, -2]


We encrypt by adding a random subset of the public key, along with some extra noise, all multiplied by 2 (so that it is zero mod 2), and adding in the message bit.

Decrypting is just taking extracting the bottom bit from the created ciphertext mod the secret key.

In [431]:
def Encrypt(pk, m):
    # random subset
    S = [a for a in range(1, tau + 1) if bool(round(random.random()))]
    r = random.randint(-(2**rho_prime) + 1, (2**rho_prime) - 1)
    c = m + 2 * r + 2 * sum(pk[i] for i in S)
    c = rem(c, pk[0])
    return c

def Decrypt(p, c):
    return rem(c, p) % 2

Here we run 1,000 random tests of the code, and print a small example.

In [ ]:
# send the message 0 or 1
failed = 0
for i in range(1000):
    m = int(round(random.random()))
    ciphertext = Encrypt(xi, m)
    decrypted = Decrypt(p, ciphertext)
    if m != decrypted:
        if not failed:
            print ('Message:', m)
            print ('Ciphertext:', ciphertext)
            print ('Decrypted to:', decrypted)
        failed += 1
if failed:
    print ('Failed:', failed)
else:
    print ('Message:', m)
    print ('Ciphertext:', ciphertext)
    print ('Decrypted to:', decrypted
)

Message: 1
Ciphertext: -7
Decrypted to: 1
